In [1]:
import openai
import os

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY

def ask_gpt(model, system_prompt, question, conversation_history=None, max_history=5):
    if conversation_history is None:
        conversation_history = []

    # Truncate conversation_history if it exceeds the limit
    if len(conversation_history) > max_history:
        conversation_history = conversation_history[-max_history:]

    conversation = [
        {"role": "system", "content": system_prompt},
        *conversation_history,
        {"role": "user", "content": question}
    ]

    response = openai.ChatCompletion.create(
        model=model,
        messages=conversation,
        max_tokens=150,                 # one token is roughly 4 characters
        temperature=1.0,
        top_p=1,
    )

    assistant_response = response['choices'][0]['message']['content']
    
    # Update the conversation history
    conversation_history.append({"role": "assistant", "content": assistant_response})
    conversation_history.append({"role": "user", "content": question})

    return assistant_response, conversation_history


In [5]:

# Usage example
model_name = "gpt-3.5-turbo"
# model_name = "gpt-4"
system_prompt = "You are a helpful assistant that provides new parents with baby name suggestions."

question = "Can you give me some categories of names that I could investigate?"
response, convo_history = ask_gpt(model_name, system_prompt, question)
print(response)


Certainly! Here are some categories of baby names that you might like to consider:

1. Classic names, such as Elizabeth and William
2. Names that reflect nature, such as River and Willow
3. Names inspired by mythology, such as Athena and Thor
4. Names that reflect your cultural background, such as Aiden and Sana
5. Unisex names, such as Quinn and Jordan
6. Names that reflect your favorite fictional characters, such as Hermione and Arya
7. Names that start with a particular letter, such as L names or M names
8. Names with a particular meaning, such as Joy or Grace
9. Names that are associated with particular colors, such as Scarlett and Raven
10. Names


In [6]:
question = "Can you suggest me a list of 10 boys names which are both classic and nature inspired?"
response, convo_history = ask_gpt(model_name, system_prompt, question)
print(response)

Sure, here are 10 classic and nature-inspired names for boys:

1. Jasper - inspired by the name of a gemstone and also after the beautiful natural park in Canada.
2. Forrest - means "woodland" and is a classic name for boys.
3. Phoenix - inspired by the mythical bird and is also the name of a city in Arizona.
4. Asher - a biblical name that means "happy" or "blessed" and also refers to trees or groves of trees.
5. River - a name that is inspired by natural waterways and is also a popular unisex name.
6. Orion - inspired by the constellation named after a hunter in Greek mythology.
7. Leo - meaning "lion", a


In [7]:
question = """ This is my list of favourite boys names: 
---
Adam, Felix, Odin, Membrillo
---
Can you please give me 10 more boys names that are similar to these?
"""
response, convo_history = ask_gpt(model_name, system_prompt, question)
print(response)

Absolutely! Here are 10 more boys names that are similar to the names on your list:

1. Leo
2. Jasper
3. Silas
4. Caspian
5. August
6. Ezra
7. Atlas
8. Phoenix
9. Leonidas
10. Orion

I hope these suggestions are helpful in finding the perfect name for your baby boy!


In [8]:
question = """ User: Can you make them sound more earthy?
---
[System] Provide a list of 10 boys names
---
"""
response, convo_history = ask_gpt(model_name, system_prompt, question)
print(response)

Sure! Here are 10 earthy boy names to consider:

1. Jasper
2. Orion
3. Sage
4. Asher
5. Cedar
6. Terra
7. River
8. Forrest
9. Phoenix
10. Canyon


In [9]:
question = """ User: I like the name Asher, can you convince me using all the best reasons why it's a great name?
"""
response, convo_history = ask_gpt(model_name, system_prompt, question)
print(response)

Absolutely! Here are some reasons why Asher is a great name:

1. It's a unique name: Asher is not as common as some other traditional baby names, so your child will stand out with this name.

2. It has a great meaning: Asher means "happy" or "blessed" in Hebrew. What better name to give to your little bundle of joy than one that means happiness and blessings?

3. It's easy to pronounce: Asher is a simple one-syllable name that doesn't require difficult pronunciation, making it easy for your child to learn to spell and pronounce their name.

4. It's versatile: Asher can be paired with a variety of middle names and last names and still sound


In [2]:
!pip install discord-components

ERROR: Could not find a version that satisfies the requirement discord-components (from versions: none)
ERROR: No matching distribution found for discord-components


In [3]:
import os
import openai
import discord
import nest_asyncio
from discord.ext import commands

# Apply the nest_asyncio patch
nest_asyncio.apply()

DISCORD_TOKEN = os.getenv('DISCORD_TOKEN_NAMEBABY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
TARGET_CHANNEL_ID = int(os.getenv('DISCORD_CHANNEL_ID_NAMEBABY'))
openai.api_key = OPENAI_API_KEY

intents = discord.Intents.default()
intents.messages = True
intents.message_content = True

class ChatBot(discord.Client):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model = "gpt-3.5-turbo"
        self.system_prompt = """You are a helpful assistant that provides new parents with baby name suggestions.
                                Include an emoji at the end of your message"""
        self.conversation_history = [] 
       
    async def on_ready(self):
        print(f'We have logged in as {self.user}')
        target_channel = self.get_channel(TARGET_CHANNEL_ID)
        if target_channel:
            await target_channel.send("`namebaby` is online  💅 Type `/help` for a list of commands.")
        else:
            print("Could not find the target channel.")

    async def on_message(self, message):
        if message.author == self.user:
            return

        print(f"incoming: {message.content}")

        if message.content.startswith("/help") or message.content.startswith("/info"):
            # respond with current config
            await message.channel.send(f"**namebaby Config**\nModel: {self.model}\n")
        
        elif message.content.startswith("/test"):
            await message.channel.send('test successful')
            
        else:
            assistant_response, self.conversation_history = ask_gpt(
                model=self.model,
                system_prompt=self.system_prompt,
                question=message.content,
                conversation_history=self.conversation_history,
                max_history=3) 
            
            print(f"response: {assistant_response}")
            await message.channel.send(assistant_response)

client = ChatBot(intents=intents)
await client.start(DISCORD_TOKEN)


We have logged in as namebaby#4590
